In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import os 

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq


from google.colab import drive
drive.mount("/content/drive")
path = '/content/drive/MyDrive/Colab Notebooks'

os.chdir(path)
!ls

file = '1661-0.txt'
text = open(file).read().lower()
print('corpus length:', len(text))

tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

print("unique_words", len(unique_words))
print("index", unique_word_index)
print("total words", len(words))

WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0], prev_words[1])
#print(next_words[0])

Mounted at /content/drive
 1661-0.gdoc			     'NEW PYTHON 3 NOTEBOOK.ipynb'
 1661-0.txt			      NextWord.ipynb
 california_cities.csv		      sample.ipynb
 Fremont_Bridge_Bicycle_Counter.csv   someplotting.ipynb
 gdp_per_capita.csv		      Untitled0.ipynb
 Kaggler.ipynb			      Untitled1.ipynb
corpus length: 581888
unique_words 8201
index {'0': 0, '000': 1, '1': 2, '10': 3, '100': 4, '1000': 5, '10_s_': 6, '10th': 7, '11': 8, '117': 9, '120': 10, '12_s_': 11, '12th': 12, '14': 13, '140': 14, '15': 15, '150': 16, '1500': 17, '1661': 18, '16a': 19, '17': 20, '1846': 21, '1858': 22, '1869': 23, '1870': 24, '1878': 25, '1883': 26, '1884': 27, '1887': 28, '1888': 29, '1890': 30, '19th': 31, '1_s_': 32, '2': 33, '20': 34, '200': 35, '2001': 36, '2002': 37, '2019': 38, '220': 39, '221b': 40, '226': 41, '22nd': 42, '249': 43, '25': 44, '250': 45, '26': 46, '26_s_': 47, '27': 48, '270': 49, '29': 50, '2_s_': 51, '2nd': 52, '3': 53, '30': 54, '31': 55, '35': 56, '3rd': 57, '4': 58, '40': 59, '4000

In [3]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
print("X:", X.shape, "Y:", Y.shape)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

print(X[0][2])

X: (109221, 5, 8201) Y: (109221, 8201)
[False False False ... False False False]


In [4]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 261s 322ms/step - loss: 6.0056 - accuracy: 0.1085 - val_loss: 7.0954 - val_accuracy: 0.1045
Epoch 2/2
811/811 [==============================] - 257s 317ms/step - loss: 5.7656 - accuracy: 0.1473 - val_loss: 7.9954 - val_accuracy: 0.1078


In [ ]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))